In [2]:
import json 
import pandas as pd
import numpy as np

In [3]:
with open(f'wikipedia-movies.json', mode='r') as file:
    wiki_movies_raw = json.load(file)

In [5]:
len(wiki_movies_raw)

7311

In [6]:
# First 5 records
wiki_movies_raw[:5]

[{'url': 'https://en.wikipedia.org/wiki/The_Adventures_of_Ford_Fairlane',
  'year': 1990,
  'imdb_link': 'https://www.imdb.com/title/tt0098987/',
  'title': 'The Adventures of Ford Fairlane',
  'Directed by': 'Renny Harlin',
  'Produced by': ['Steve Perry', 'Joel Silver'],
  'Screenplay by': ['David Arnott', 'James Cappe', 'Daniel Waters'],
  'Story by': ['David Arnott', 'James Cappe'],
  'Based on': ['Characters', 'by Rex Weiner'],
  'Starring': ['Andrew Dice Clay',
   'Wayne Newton',
   'Priscilla Presley',
   'Lauren Holly',
   'Morris Day',
   'Robert Englund',
   "Ed O'Neill"],
  'Narrated by': 'Andrew "Dice" Clay',
  'Music by': ['Cliff Eidelman', 'Yello'],
  'Cinematography': 'Oliver Wood',
  'Edited by': 'Michael Tronick',
  'Productioncompany ': 'Silver Pictures',
  'Distributed by': '20th Century Fox',
  'Release date': ['July 11, 1990', '(', '1990-07-11', ')'],
  'Running time': '102 minutes',
  'Country': 'United States',
  'Language': 'English',
  'Budget': '$20 million',


In [9]:
# Last 5 records
wiki_movies_raw[-5:]

[{'url': 'https://en.wikipedia.org/wiki/Holmes_%26_Watson',
  'year': 2018,
  'imdb_link': 'https://www.imdb.com/title/tt1255919/',
  'title': 'Holmes & Watson',
  'Directed by': 'Etan Cohen',
  'Produced by': ['Will Ferrell',
   'Adam McKay',
   'Jimmy Miller',
   'Clayton Townsend'],
  'Screenplay by': 'Etan Cohen',
  'Based on': ['Sherlock Holmes',
   'and',
   'Dr. Watson',
   'by',
   'Sir Arthur Conan Doyle'],
  'Starring': ['Will Ferrell',
   'John C. Reilly',
   'Rebecca Hall',
   'Rob Brydon',
   'Steve Coogan',
   'Ralph Fiennes'],
  'Music by': 'Mark Mothersbaugh',
  'Cinematography': 'Oliver Wood',
  'Edited by': 'Dean Zimmerman',
  'Productioncompanies ': ['Columbia Pictures',
   'Gary Sanchez Productions',
   'Mosaic Media Group',
   'Mimran Schur Pictures'],
  'Distributed by': 'Sony Pictures Releasing',
  'Release date': ['December 25, 2018',
   '(',
   '2018-12-25',
   ')',
   '(United States)'],
  'Running time': '90 minutes',
  'Country': 'United States',
  'Language

In [10]:
# Some records in the middle
wiki_movies_raw[3600:3605]

[{'url': 'https://en.wikipedia.org/wiki/Benji:_Off_the_Leash!',
  'year': 2004,
  'imdb_link': 'https://www.imdb.com/title/tt0315273/',
  'title': 'Benji: Off the Leash!',
  'Directed by': 'Joe Camp',
  'Written by': 'Joe Camp',
  'Starring': ['Benji', 'Nick Whitaker', 'Shaggy', 'Gypsy the Cockatoo'],
  'Music by': 'Antonio di Lorenzo',
  'Productioncompany ': 'Mulberry Square Productions',
  'Distributed by': 'Mulberry Square Productions',
  'Release date': ['March 26, 2004', '(', '2004-03-26', ')'],
  'Running time': '97 min',
  'Country': 'United States',
  'Language': 'English',
  'Box office': '$3,817,362'},
 {'url': 'https://en.wikipedia.org/wiki/The_Best_Thief_in_the_World',
  'year': 2004,
  'imdb_link': 'https://www.imdb.com/title/tt0389796/',
  'title': 'The Best Thief in the World',
  'Directed by': 'Jacob Kornbluth',
  'Produced by': ['Tim Perrell', 'Nicola Usborne'],
  'Written by': 'Jacob Kornbluth',
  'Starring': ['Marc Rozendaal',
   'Michael Silverman',
   'David Warsh

In [13]:
kaggle_metadata = pd.read_csv("movies_metadata.csv", low_memory = False)
ratings = pd.read_csv("ratings.csv")

In [14]:
kaggle_metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [15]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [19]:
ratings.sample(5)

,userId,movieId,rating,timestamp
5111046,52643,8983,4.0,1459548433
14071529,146185,2012,2.0,974743183
25803989,268458,2003,3.0,1095569076
25645993,266870,541,3.0,1461398168
11537998,119696,33004,4.0,1294434301


In [18]:
kaggle_metadata.sample(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
17971,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",NaN,27999,tt0049405,en,The Killer Is Loose,"A savings-and-loan bank is robbed; later, a po...",...,1956-03-02,0.0,73.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,He was no ordinary killer... She was no ordina...,The Killer Is Loose,False,7.2,5.0
6793,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,11846,tt0101862,en,Father of the Bride,"George Banks is an ordinary, middle-class man ...",...,1991-12-20,89325780.0,105.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Love is wonderful. Until it happens to your on...,Father of the Bride,False,6.2,355.0
33367,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,67337,tt0063183,en,Zhui hun biao,"They brutally killed his wife, wiped out dozen...",...,1968-05-09,0.0,87.0,"[{'iso_639_1': 'zh', 'name': '普通话'}]",Released,NaN,Killer Darts,False,6.3,6.0
40036,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,381418,tt5255710,hi,Sanam Teri Kasam,A musical romantic movie exploring the journey...,...,2016-02-05,0.0,154.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Sanam Teri Kasam,False,7.6,24.0
33132,False,NaN,0,"[{'id': 10769, 'name': 'Foreign'}, {'id': 18, ...",NaN,66447,tt0108290,fr,Tango,NaN,...,1993-02-03,0.0,88.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,NaN,Tango,False,4.4,4.0


In [22]:
wiki_movies_df = pd.DataFrame(wiki_movies_raw)
wiki_movies_df.sample(15)

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Predecessor,Founders,Area served,Products,Services,Russian,Hebrew,Revenue,Operating income,Polish
2700,https://en.wikipedia.org/wiki/The_Art_of_War_(...,2000.0,https://www.imdb.com/title/tt0160009/,The Art of War,Christian Duguay,Nicolas Clermont,"[Wayne Beach, Simon Barry]",Wayne Beach,NaN,"[Wesley Snipes, Anne Archer, Maury Chaykin, Ma...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
947,https://en.wikipedia.org/wiki/Class_of_Nuke_%2...,1994.0,https://www.imdb.com/title/tt0112685/,"Class of Nuke 'Em High 3: The Good, the Bad an...",Eric Louzil,"[Michael Herz, Lloyd Kaufman]","[Stephen Gerard, Lloyd Kaufman, Eric Louzil, C...","[Lloyd Kaufman, Carl Morano, Matt Unger]",NaN,"[Brick Bronsky, Lisa Star, John Tallman, Lisa ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4731,https://en.wikipedia.org/wiki/Sleuth_(2007_film),2007.0,https://www.imdb.com/title/tt0857265/,Sleuth,Kenneth Branagh,"[Kenneth Branagh, Simon Halfon, Jude Law, Simo...",Harold Pinter,NaN,"[Sleuth, by, Anthony Shaffer]","[Michael Caine, Jude Law]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6585,https://en.wikipedia.org/wiki/The_Longest_Ride...,2015.0,https://www.imdb.com/title/tt2726560/,The Longest Ride,George Tillman Jr.,"[Marty Bowen, Wyck Godfrey, Theresa Park, Nich...",Craig Bolotin,NaN,"[The Longest Ride, by Nicholas Sparks]","[Scott Eastwood, Britt Robertson, Jack Huston,...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4035,https://en.wikipedia.org/wiki/The_Upside_of_Anger,2005.0,https://www.imdb.com/title/tt0365885/,The Upside of Anger,Mike Binder,"[Jack Binder, Alex Gartner, Sammy Lee]",NaN,NaN,NaN,"[Joan Allen, Kevin Costner, Erika Christensen,...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1297,https://en.wikipedia.org/wiki/The_Gate_of_Heav...,1995.0,NaN,Transcriptions,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
474,https://en.wikipedia.org/wiki/Aladdin_(1992_Di...,1992.0,https://www.imdb.com/title/tt0103639/,Aladdin,"[Ron Clements, John Musker]","[Ron Clements, John Musker]","[Ron Clements, John Musker, Ted Elliott, Terry...","[Burny Mattinson, Roger Allers, Daan Jippes, K...","[Aladdin and the Magic Lamp, from, One Thousan...","[Scott Weinger, Robin Williams, Linda Larkin, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2663,https://en.wikipedia.org/wiki/The_Thomas_Crown...,1999.0,https://www.imdb.com/title/tt0155267/,The Thomas Crown Affair,John McTiernan,"[Michael Tadross, Pierce Brosnan, Beau St. Clair]","[Leslie Dixon, Kurt Wimmer]",Alan Trustman,NaN,"[Pierce Brosnan, Rene Russo, Denis Leary]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5843,https://en.wikipedia.org/wiki/Moonrise_Kingdom,2012.0,https://www.imdb.com/title/tt1748122/,Moonrise Kingdom,Wes Anderson,"[Wes Anderson, Scott Rudin, Steven Rales, Jere...",NaN,NaN,NaN,"[Bruce Willis, Edward Norton, Bill Murray, Fra...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
842,https://en.wikipedia.org/wiki/Roosters,1993.0,https://www.imdb.com/title/tt0107987/,Roosters,Robert M. Young,"[Susan Block-Reiner, Norman I. Cohen, Kevin Re...",NaN,NaN,NaN,"[Edward James Olmos, Sônia Braga, María Conchi...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
#wiki_movies_df.columns --> does not produce the entire list
wiki_movies_df.columns.to_list()

['url',
 'year',
 'imdb_link',
 'title',
 'Directed by',
 'Produced by',
 'Screenplay by',
 'Story by',
 'Based on',
 'Starring',
 'Narrated by',
 'Music by',
 'Cinematography',
 'Edited by',
 'Productioncompany ',
 'Distributed by',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Written by',
 'Genre',
 'Theme music composer',
 'Country of origin',
 'Original language(s)',
 'Producer(s)',
 'Editor(s)',
 'Production company(s)',
 'Original network',
 'Original release',
 'Productioncompanies ',
 'Executive producer(s)',
 'Production location(s)',
 'Distributor',
 'Picture format',
 'Audio format',
 'Voices of',
 'Followed by',
 'Composer(s)',
 'Created by',
 'Also known as',
 'Opening theme',
 'No. of episodes',
 'Preceded by',
 'Author',
 'Publisher',
 'Publication date',
 'Media type',
 'Pages',
 'ISBN',
 'OCLC',
 'LC Class',
 'Cover artist',
 'Series',
 'Set in',
 'Adaptation by',
 'Suggested by',
 'Biographical data',
 'Born',
 'Died',
 'Resti

In [30]:
wiki_movies_raw[:1]

[{'url': 'https://en.wikipedia.org/wiki/The_Adventures_of_Ford_Fairlane',
  'year': 1990,
  'imdb_link': 'https://www.imdb.com/title/tt0098987/',
  'title': 'The Adventures of Ford Fairlane',
  'Directed by': 'Renny Harlin',
  'Produced by': ['Steve Perry', 'Joel Silver'],
  'Screenplay by': ['David Arnott', 'James Cappe', 'Daniel Waters'],
  'Story by': ['David Arnott', 'James Cappe'],
  'Based on': ['Characters', 'by Rex Weiner'],
  'Starring': ['Andrew Dice Clay',
   'Wayne Newton',
   'Priscilla Presley',
   'Lauren Holly',
   'Morris Day',
   'Robert Englund',
   "Ed O'Neill"],
  'Narrated by': 'Andrew "Dice" Clay',
  'Music by': ['Cliff Eidelman', 'Yello'],
  'Cinematography': 'Oliver Wood',
  'Edited by': 'Michael Tronick',
  'Productioncompany ': 'Silver Pictures',
  'Distributed by': '20th Century Fox',
  'Release date': ['July 11, 1990', '(', '1990-07-11', ')'],
  'Running time': '102 minutes',
  'Country': 'United States',
  'Language': 'English',
  'Budget': '$20 million',


In [37]:
wiki_movies = [movie for movie in wiki_movies_raw 
               if ('Director' in movie or 'Directed by' in movie) and ('imdb_link' in movie)
              ]
len(wiki_movies)

7080

In [40]:
wiki_movies_df = pd.DataFrame(wiki_movies)
wiki_movies_df.head()

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Hepburn,Literally,Cantonese,Chinese,Yiddish,Arabic,Romanized,Russian,Hebrew,Polish
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,Renny Harlin,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet",James Foley,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,Roger Spottiswoode,Daniel Melnick,"[John Eskow, Richard Rush]",NaN,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,Woody Allen,Robert Greenhut,NaN,NaN,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,John Cornell,John Cornell,NaN,NaN,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
# Seems like we have an column 'no. episode' where it indicate it is a TV show, but not a movie. 
wiki_movies_df.columns.to_list()

['url',
 'year',
 'imdb_link',
 'title',
 'Directed by',
 'Produced by',
 'Screenplay by',
 'Story by',
 'Based on',
 'Starring',
 'Narrated by',
 'Music by',
 'Cinematography',
 'Edited by',
 'Productioncompany ',
 'Distributed by',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Written by',
 'Genre',
 'Theme music composer',
 'Country of origin',
 'Original language(s)',
 'Producer(s)',
 'Editor(s)',
 'Production company(s)',
 'Original network',
 'Original release',
 'Productioncompanies ',
 'Executive producer(s)',
 'Production location(s)',
 'Distributor',
 'Picture format',
 'Audio format',
 'Voices of',
 'Followed by',
 'Composer(s)',
 'Created by',
 'Also known as',
 'Opening theme',
 'No. of episodes',
 'Preceded by',
 'Adaptation by',
 'Suggested by',
 'Traditional',
 'Mandarin',
 'Released',
 'Recorded',
 'Venue',
 'Length',
 'Label',
 'Director',
 'Producer',
 'Animation by',
 'Color process',
 'Simplified',
 'French',
 'Camera setup'

In [45]:
wiki_movies = [movie for movie in wiki_movies_raw 
               if (('Director' in movie or 'Directed by' in movie) and ('imdb_link' in movie) and ('No. of episodes' not in movie))
              ]
len(wiki_movies)

7076

In [136]:
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    
    alt_titles = {}
    alt_titles_list = ['Also known as', 'Arabic', 'Cantonese', 'Chinese', 'French', 'Hangul', 'Hebrew', 'Hepburn', 'Japanese', 'Literally',  'Mandarin', 'McCune–Reischauer', 'Original title', 'Polish',  'Revised Romanization', 'Romanized', 'Russian', 'Simplified', 'Traditional', 'Yiddish']
    
    for key in alt_titles_list:
        if key in movie: 
            alt_titles[key] = movie[key]
            movie.pop(key)
        if len(alt_titles) > 0:
            movie['alt_titles'] = alt_titles
    
    
    return movie

In [124]:
clean_movie(wiki_movies[6632])

{'url': 'https://en.wikipedia.org/wiki/Burning_Sands_(2017_film)',
 'year': 2017,
 'imdb_link': 'https://www.imdb.com/title/tt0005826432/',
 'title': 'Burning Sands',
 'Directed by': 'Gerard McMurray',
 'Produced by': ['Stephanie Allain',
  'Jason Michael Berman',
  'Reginald Hudlin',
  'Mel Jones',
  'Camila Tanabe',
  '(assoc.)',
  'Patrick Raymond',
  '(assoc.)'],
 'Written by': ['Gerard McMurray', 'Christine Berg'],
 'Starring': ['Trevor Jackson',
  'Alfre Woodard',
  'Steve Harris',
  'Tosin Cole',
  'DeRon Horton',
  'Christian Robinson',
  'Trevante Rhodes',
  'Jay Q Saun',
  'Serayah'],
 'Music by': 'Kevin Lax',
 'Cinematography': 'Isiah Donté Lee',
 'Edited by': 'Evan Schrodek',
 'Productioncompany ': ['Mandalay Pictures',
  'Homegrown Pictures',
  'Hudlin Entertainment',
  'Freedom Road Productions'],
 'Distributed by': 'Netflix',
 'Release date': ['January 24, 2017',
  '(',
  '2017-01-24',
  ')',
  '(',
  'Sundance',
  ')',
  'March 10, 2017',
  '(',
  '2017-03-10',
  ')',
 

In [50]:
# Data with values in Arabic column.
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Hepburn,Literally,Cantonese,Chinese,Yiddish,Arabic,Romanized,Russian,Hebrew,Polish
6838,https://en.wikipedia.org/wiki/The_Insult_(film),2018,https://www.imdb.com/title/tt7048622/,The Insult,Ziad Doueiri,"[Rachid Bouchareb, Jean Bréhat, Julie Gayet, A...",NaN,NaN,NaN,"[Adel Karam, Kamel El Basha]",...,NaN,Case No. 23,NaN,NaN,NaN,قضية رقم ٢٣,Qadiyya raqm 23,NaN,NaN,NaN
7062,https://en.wikipedia.org/wiki/Capernaum_(film),2018,https://www.imdb.com/title/tt8267604/,Capernaum,Nadine Labaki,"[Michel Merkt, Khaled Mouzanar]","[Nadine Labaki, Jihad Hojaily, Michelle Keserw...","[Georges Khabbaz, Nadine Labaki, Michelle Kese...",NaN,"[Zain Al Rafeea, Yordanos Shiferaw, Boluwatife...",...,NaN,NaN,NaN,NaN,NaN,کفرناحوم‎,NaN,NaN,NaN,NaN


In [54]:
# Getting urls for both Arabic movies.
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]['url']

6838    https://en.wikipedia.org/wiki/The_Insult_(film)
7062     https://en.wikipedia.org/wiki/Capernaum_(film)
Name: url, dtype: object

In [57]:
sorted(wiki_movies_df.columns.tolist())

['Adaptation by',
 'Also known as',
 'Animation by',
 'Arabic',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Camera setup',
 'Cantonese',
 'Chinese',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Country of origin',
 'Created by',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Edited by',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'French',
 'Genre',
 'Hangul',
 'Hebrew',
 'Hepburn',
 'Japanese',
 'Label',
 'Language',
 'Length',
 'Literally',
 'Mandarin',
 'McCune–Reischauer',
 'Music by',
 'Narrated by',
 'No. of episodes',
 'Opening theme',
 'Original language(s)',
 'Original network',
 'Original release',
 'Original title',
 'Picture format',
 'Polish',
 'Preceded by',
 'Produced by',
 'Producer',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Productioncompanies ',
 'Productioncompany ',
 'Recorded',
 'Release date',
 'Released',
 'Revised Romanization',
 'Romanized',
 'Running time',
 'Russ

In [69]:
wiki_movies_df[wiki_movies_df['Chinese'].notnull()]['Chinese']

6624    摇滚藏獒
Name: Chinese, dtype: object

In [70]:
wiki_movies_df[wiki_movies_df['title'].notnull()]['title'][6624]

'Rock Dog'

In [71]:
# Here is a complete list of columns that hold alternate title data: Also known as, Arabic, Cantonese, Chinese, French, Hangul, Hebrew, Hepburn, 
# Japanese, Literally,  Mandarin, McCune–Reischauer, Original title, Polish,  Revised Romanization, Romanized, Russian, Simplified, Traditional, 
# Yiddish


In [137]:
clean_movies = [ clean_movie(movie) for movie in wiki_movies]

In [139]:
wiki_movies_df = pd.DataFrame(clean_movies)

In [143]:
wiki_movies_df.columns.to_list()

['url',
 'year',
 'imdb_link',
 'title',
 'Directed by',
 'Produced by',
 'Screenplay by',
 'Story by',
 'Based on',
 'Starring',
 'Narrated by',
 'Music by',
 'Cinematography',
 'Edited by',
 'Productioncompany ',
 'Distributed by',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Written by',
 'Genre',
 'Theme music composer',
 'Country of origin',
 'Original language(s)',
 'Producer(s)',
 'Editor(s)',
 'Production company(s)',
 'Original network',
 'Original release',
 'Productioncompanies ',
 'Executive producer(s)',
 'Production location(s)',
 'Distributor',
 'Picture format',
 'Audio format',
 'Voices of',
 'Followed by',
 'Composer(s)',
 'Created by',
 'Preceded by',
 'Adaptation by',
 'Suggested by',
 'alt_titles',
 'Released',
 'Recorded',
 'Venue',
 'Length',
 'Label',
 'Director',
 'Producer',
 'Animation by',
 'Color process',
 'Screen story by']

In [151]:
wiki_movies_df[wiki_movies_df['alt_titles'].notnull()]['alt_titles']

858            {'Mandarin': 'Xǐyàn', 'Traditional': '喜宴'}
1342       {'Also known as': 'Detonator II: Night Watch'}
1534    {'Mandarin': 'Dōng Gōng Xī Gōng', 'Simplified'...
1888                   {'French': 'Le Cinquième Élément'}
3413    {'Hangul': '원더풀 데이즈', 'McCune–Reischauer': 'Wŏ...
4238    {'Hangul': '파이 스토리', 'McCune–Reischauer': 'P‘a...
4790                  {'Original title': 'ואלס עם באשיר'}
6393    {'Hepburn': 'Omoide no Mānī', 'Japanese': '思い出...
6456    {'Cantonese': ['Jip', '6', 'Man', '6', 'Saam',...
6613    {'Mandarin': 'Cháng Chéng', 'Simplified': '长城'...
6620                                  {'Chinese': '摇滚藏獒'}
6661    {'Simplified': '我们诞生在中国', 'Traditional': '我們誕生...
6719                                  {'Yiddish': 'מנשה'}
6829    {'Hepburn': 'Meari to Majo no Hana', 'Japanese...
6834    {'Arabic': 'قضية رقم ٢٣', 'Literally': 'Case N...
6849     {'Romanized': 'Nelyubov', 'Russian': 'Нелюбовь'}
6862                           {'Hebrew': 'פוֹקְסטְרוֹט'}
7009    {'Hepb